In [2]:
!pip install transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 39.6 MB/s eta 0:00:0000:0100:01




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.8 MB/s eta 0:00:00








     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00











In [34]:
import torch

import numpy as np
from transformers import AutoTokenizer, AutoModel

class TransformerVectorizer:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def get_entry_vectors(self, entries):
        entry_vectors = []
        for entry in entries:
            sentence_vectors = []
            for sentence in entry:
                encoded_input = self.tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
                with torch.no_grad():
                    model_output = self.model(**encoded_input)
                sentence_vector = model_output.last_hidden_state.mean(dim=1).squeeze().numpy()
                sentence_vectors.append(sentence_vector)
            entry_vector = np.mean(sentence_vectors, axis=0)
            entry_vectors.append(entry_vector.reshape(1, -1))  # Reshape to (1, 768)
        return entry_vectors

# Example usage
dataset = [
    ["This is the first sentence in entry 1.", "Another sentence follows."],
    ["The second entry has multiple sentences.", "This is the first.", "This is the second."],
    ["Entry 3 has only one sentence."]
]

model_name = "bert-base-uncased"  # Specify the desired transformer model
vectorizer = TransformerVectorizer(model_name)
vectors = vectorizer.get_entry_vectors(dataset)

#for i, vector in enumerate(vectors):
   # print(f"Entry {i+1} vector shape:", vector.shape)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']

- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
len(vectors)

3

In [36]:
vectors[0].shape

(1, 768)

In [38]:
vectors[2].shape

(1, 768)

In [61]:
import pandas as pd
df=pd.DataFrame()

In [58]:
#df['vector_column']=vectors

In [59]:
df

,vector_column
0,"[[-0.056468755, -0.093506224, 0.013101123, -0...."
1,"[[-0.11627471, -0.24916555, 0.22335304, -0.173..."
2,"[[-0.21733417, 0.13716565, -0.13622428, 0.0764..."


In [48]:
type(df["text_vectors"][0])


numpy.ndarray

In [53]:

# Assuming you have the vector representations in a DataFrame called 'df'

# Reshape the array column to 2D input
#df['vector_column'] = df['vector_column'].apply(lambda x: np.squeeze(x))
#df

,vector_column
0,"[-0.056468755, -0.093506224, 0.013101123, -0.1..."
1,"[-0.11627471, -0.24916555, 0.22335304, -0.1731..."
2,"[-0.21733417, 0.13716565, -0.13622428, 0.07648..."


In [60]:
(df["vector_column"][0]).shape


(1, 768)

In [62]:
### To Convert each vectors into 768 dimesnions columns

for i in range(0,len(vectors)):
   df=df.append(pd.DataFrame(vectors[i].reshape(1,-1)),ignore_index=True)

<ipython-input-62-19c53f893f9b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

  df=df.append(pd.DataFrame(vectors[i].reshape(1,-1)),ignore_index=True)


In [63]:
df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.056469,-0.093506,0.013101,-0.172887,0.197085,-0.205753,0.535066,0.544561,0.015874,-0.243412,...,0.198407,-0.047856,-0.100783,-0.169452,-0.148156,0.418461,0.658204,0.056111,0.090105,0.154940
1,-0.116275,-0.249166,0.223353,-0.173110,0.081982,-0.111024,0.132601,0.325200,-0.016477,-0.533564,...,0.223735,0.029769,0.029928,-0.136823,-0.186936,0.002034,0.241519,-0.007107,0.353113,0.083695
2,-0.217334,0.137166,-0.136224,0.076490,-0.008358,-0.086810,0.293742,0.073594,-0.081530,-0.053138,...,0.372527,-0.127273,-0.066425,-0.163427,-0.003627,-0.062237,0.512631,0.203284,0.094521,0.216966
